<a href="https://colab.research.google.com/github/monoramasn/Speech_fairness/blob/main/hr_whisper_base_test_(edit).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
!pip install gradio

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-6rv4_3c6
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-6rv4_3c6
  Resolved https://github.com/huggingface/transformers to commit ffa04def0e6765cdf1d9518546506325a72e2bbe
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.37.0.dev0-py3-none-any.whl size=8269431 sha256=7f2621a9c86d7b7dd96cc68299255941f8cf2707204598b3b152a6c8ec79007b
  Stored in directory: /tmp/pip-ephem-wheel-cache-2ztz213z/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import torch
from transformers import AutoFeatureExtractor, WhisperModel
from datasets import load_dataset

In [3]:
import os
import numpy as np

try:
    import tensorflow  # required in Colab to avoid protobuf compatibility issues
except ImportError:
    pass

import torch
import pandas as pd
import torchaudio

from tqdm.notebook import tqdm

In [58]:
voxpopuli_data = load_dataset("facebook/voxpopuli", "hr", split="test")
#column_name = "audio, normalized_text"

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [60]:
voxpopuli_data

Dataset({
    features: ['audio_id', 'language', 'audio', 'raw_text', 'normalized_text', 'gender', 'speaker_id', 'is_gold_transcript', 'accent'],
    num_rows: 666
})

In [62]:
!pip install openai-whisper
import whisper
#model = whisper.load_model("base")

In [64]:
model = whisper.load_model("base")

In [65]:
from transformers import WhisperForConditionalGeneration, WhisperFeatureExtractor,WhisperTokenizer
from datasets import load_dataset
import torch
from tqdm import tqdm
from jiwer import wer

In [66]:
Gender=voxpopuli_data['gender']
Audio=voxpopuli_data['audio']
text=voxpopuli_data['normalized_text']
Gender=pd.DataFrame(Gender)
Audio=pd.DataFrame(Audio)
text=pd.DataFrame(text)

In [67]:
text

,0
0,na primjer omogućila je mnogim talentiranim lj...
1,potovani predsjedavajui ribarstvo je vana gosp...
2,nastavlja na temelju ovog europskog projekta k...
3,gospoo predsjedavajua dananje izvjee o europsk...
4,bitno je doprinijeti većem povjerenju potrošač...
...,...
661,radi se o čestim blokadama vlasti napuštanju s...
662,napad islamistikih militanata u sjevernoj nige...
663,ne smijemo ostati slijepi na činjenicu da i eu...
664,u mnogim zemljama ako niste uhvaćeni u počinje...


In [68]:
Gender.columns=['Gender']
text.columns=['Transcript']

In [69]:
file=pd.concat([Gender, text, Audio],axis=1)
file

,Gender,Transcript,path,array,sampling_rate
0,female,na primjer omogućila je mnogim talentiranim lj...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.00286865234375, 0.003021240234375, 0.002655...",16000
1,male,potovani predsjedavajui ribarstvo je vana gosp...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.000244140625, 0.000152587890625, -0.0006408...",16000
2,male,nastavlja na temelju ovog europskog projekta k...,/root/.cache/huggingface/datasets/downloads/ex...,"[6.103515625e-05, -0.00018310546875, 9.1552734...",16000
3,male,gospoo predsjedavajua dananje izvjee o europsk...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.00360107421875, 0.002685546875, -0.00036621...",16000
4,female,bitno je doprinijeti većem povjerenju potrošač...,/root/.cache/huggingface/datasets/downloads/ex...,"[-3.0517578125e-05, 0.001190185546875, 0.00189...",16000
...,...,...,...,...,...
661,male,radi se o čestim blokadama vlasti napuštanju s...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.0023193359375, -0.0010986328125, -0.0001831...",16000
662,male,napad islamistikih militanata u sjevernoj nige...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.039825439453125, 0.027557373046875, 0.02493...",16000
663,female,ne smijemo ostati slijepi na činjenicu da i eu...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.007049560546875, 0.001708984375, -0.0053100...",16000
664,male,u mnogim zemljama ako niste uhvaćeni u počinje...,/root/.cache/huggingface/datasets/downloads/ex...,"[-3.0517578125e-05, 0.000274658203125, 0.00036...",16000


In [70]:
male_samples=file.loc[file['Gender'] == 'male']
male_samples

,Gender,Transcript,path,array,sampling_rate
1,male,potovani predsjedavajui ribarstvo je vana gosp...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.000244140625, 0.000152587890625, -0.0006408...",16000
2,male,nastavlja na temelju ovog europskog projekta k...,/root/.cache/huggingface/datasets/downloads/ex...,"[6.103515625e-05, -0.00018310546875, 9.1552734...",16000
3,male,gospoo predsjedavajua dananje izvjee o europsk...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.00360107421875, 0.002685546875, -0.00036621...",16000
5,male,takva je pozicija neizbjena ne samo zbog injen...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.007781982421875, -0.009033203125, -0.00741...",16000
6,male,uloga europe mora i dalje biti promicanje ljud...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.004669189453125, -0.00543212890625, -0.003...",16000
...,...,...,...,...,...
658,male,stoga sam uvjeren da upravo visoki standardi z...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.009552001953125, -0.018829345703125, -0.01...",16000
659,male,od trenutka pristupanja hrvatske u punopravno ...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.00042724609375, 0.005462646484375, 0.00509...",16000
661,male,radi se o čestim blokadama vlasti napuštanju s...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.0023193359375, -0.0010986328125, -0.0001831...",16000
662,male,napad islamistikih militanata u sjevernoj nige...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.039825439453125, 0.027557373046875, 0.02493...",16000


In [52]:
female_samples=file.loc[file['Gender'] == 'female']
female_samples

,Gender,Transcript,path,array,sampling_rate
8,female,nyt olisi ehkä puheenjohtajakauden tilausta si...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.0059814453125, -0.00238037109375, -0.00048...",16000
9,female,näitä investointeja on tehtävä kaikissa jäsenv...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.0030517578125, -0.0003662109375, 0.0010375...",16000
11,female,arvoisa puhemies hyvät kollegat tuen talous ja...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.000152587890625, 0.000152587890625, -0.000...",16000
22,female,vihreät olisivat halunneet viedä työntekijöide...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.08245849609375, -0.0870361328125, -0.08416...",16000
24,female,tähän on ehdottomasti puututtava koska euroopp...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.0069580078125, -0.0037841796875, 6.1035156...",16000
...,...,...,...,...,...
460,female,tiedämme että lähtökohta oli huono koska komis...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.0, 0.000762939453125, -9.1552734375e-05, -0...",16000
462,female,arvoisa puhemies venäjä on eu n tärkeä kauppak...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.0009765625, -0.001678466796875, -0.0008850...",16000
469,female,nyt kahdeksan vuotta myöhemmin kreikalla on ve...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.00439453125, -0.005279541015625, 0.0051879...",16000
472,female,on välttämätöntä että myös unionin rahankäyttö...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.003936767578125, -0.001953125, 0.000457763...",16000


In [71]:
def transcribe_audio(file_path):
    result = model.transcribe(file_path, language="hr", fp16=False)
    return result["text"]

In [72]:
def calculate_wer(ground_truths, hypotheses):
    return wer(ground_truths, hypotheses)

In [73]:
from jiwer import wer
from collections import Counter

def calculate_precision_recall_f1(ground_truth, hypothesis):
    ground_truth_words = set(ground_truth.split())
    hypothesis_words = set(hypothesis.split())

    true_positives = ground_truth_words.intersection(hypothesis_words)
    false_positives = hypothesis_words - ground_truth_words
    false_negatives = ground_truth_words - hypothesis_words

    precision = len(true_positives) / (len(true_positives) + len(false_positives)) if true_positives or false_positives else 0
    recall = len(true_positives) / (len(true_positives) + len(false_negatives)) if true_positives or false_negatives else 0
    f1_score = 2 * (precision * recall) / (precision + recall) if precision + recall else 0

    return precision, recall, f1_score

In [56]:

total_wer = 0
total_precision = 0
total_recall = 0
total_f1 = 0
num_sample = 0

for index,row in male_samples.iterrows():
    audio_path=row['path']
    ground_truth_text = row['Transcript']
    transcription = transcribe_audio(audio_path)
    # Compute metrics
    sample_wer = calculate_wer(ground_truth_text, transcription)
    precision, recall, f1_score = calculate_precision_recall_f1(ground_truth_text, transcription)

    total_wer += sample_wer
    total_precision += precision
    total_recall += recall
    total_f1 += f1_score
    num_sample += 1

    print(f"Processed {num_sample} files, WER: {sample_wer}, Precision: {precision}, Recall: {recall}, F1: {f1_score}")

# Calculate average metrics
average_wer = total_wer / num_sample
average_precision = total_precision / num_sample
average_recall = total_recall / num_sample
average_f1 = total_f1 / num_sample

print(f"Average WER: {average_wer}")
print(f"Average Precision: {average_precision}")
print(f"Average Recall: {average_recall}")
print(f"Average F1-Score: {average_f1}")

Processed 1 files, WER: 0.7, Precision: 0.3, Recall: 0.3, F1: 0.3
Processed 2 files, WER: 0.4, Precision: 0.6, Recall: 0.6, F1: 0.6
Processed 3 files, WER: 0.8421052631578947, Precision: 0.3, Recall: 0.3157894736842105, F1: 0.3076923076923077
Processed 4 files, WER: 1.0, Precision: 0.0, Recall: 0.0, F1: 0
Processed 5 files, WER: 0.5384615384615384, Precision: 0.5333333333333333, Recall: 0.6153846153846154, F1: 0.5714285714285715
Processed 6 files, WER: 0.5882352941176471, Precision: 0.4375, Recall: 0.4117647058823529, F1: 0.42424242424242425
Processed 7 files, WER: 0.6296296296296297, Precision: 0.44, Recall: 0.4074074074074074, F1: 0.4230769230769231
Processed 8 files, WER: 0.5384615384615384, Precision: 0.5, Recall: 0.46153846153846156, F1: 0.48000000000000004
Processed 9 files, WER: 0.7692307692307693, Precision: 0.38461538461538464, Recall: 0.4166666666666667, F1: 0.4
Processed 10 files, WER: 0.5555555555555556, Precision: 0.5555555555555556, Recall: 0.5555555555555556, F1: 0.55555

In [75]:
total_wer = 0
total_precision = 0
total_recall = 0
total_f1 = 0
num_sample = 0

for index,row in female_samples.iterrows():
    audio_path=row['path']
    ground_truth_text = row['Transcript']
    transcription = transcribe_audio(audio_path)
    # Compute metrics
    sample_wer = calculate_wer(ground_truth_text, transcription)
    precision, recall, f1_score = calculate_precision_recall_f1(ground_truth_text, transcription)


    total_wer += sample_wer
    total_precision += precision
    total_recall += recall
    total_f1 += f1_score
    num_sample += 1

    print(f"Processed {num_sample} files, WER: {sample_wer}, Precision: {precision}, Recall: {recall}, F1: {f1_score}")

# Calculate average metrics
average_wer = total_wer / num_sample
average_precision = total_precision / num_sample
average_recall = total_recall / num_sample
average_f1 = total_f1 / num_sample

print(f"Average WER: {average_wer}")
print(f"Average Precision: {average_precision}")
print(f"Average Recall: {average_recall}")
print(f"Average F1-Score: {average_f1}")

Processed 1 files, WER: 1.4, Precision: 0.08695652173913043, Recall: 0.13333333333333333, F1: 0.10526315789473685
Processed 2 files, WER: 1.0, Precision: 0.14285714285714285, Recall: 0.16666666666666666, F1: 0.15384615384615383
Processed 3 files, WER: 0.9166666666666666, Precision: 0.14285714285714285, Recall: 0.08333333333333333, F1: 0.10526315789473685
Processed 4 files, WER: 1.3333333333333333, Precision: 0.047619047619047616, Recall: 0.06666666666666667, F1: 0.05555555555555555
Processed 5 files, WER: 1.1538461538461537, Precision: 0.06666666666666667, Recall: 0.07692307692307693, F1: 0.07142857142857144
Processed 6 files, WER: 1.2352941176470589, Precision: 0.19047619047619047, Recall: 0.23529411764705882, F1: 0.21052631578947367
Processed 7 files, WER: 1.0625, Precision: 0.15789473684210525, Recall: 0.1875, F1: 0.17142857142857143
Processed 8 files, WER: 1.2307692307692308, Precision: 0.125, Recall: 0.15384615384615385, F1: 0.13793103448275862
Processed 9 files, WER: 1.3888888888